In [ ]:
import torch, time, copy
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.quantization import prepare, convert
from src.utils import *
from src.override_resnet import *


model = resnet50_quan(weights=pretrained_weights_mapping[50])
model.to("cuda")
model.eval()

# set fuse ############################################################
model = fuse_ALL(model)

model.qconfig = torch.quantization.QConfig(
    activation=torch.quantization.HistogramObserver.with_args(reduce_range=True),
    weight=torch.quantization.PerChannelMinMaxObserver.with_args(dtype=torch.qint8),
)
prepare(model, inplace=True)

# calibrate the model ############################################################
criterion = nn.CrossEntropyLoss()
train_loader, _ = GetDataset(
    dataset_name="ImageNet",
    device="cuda",
    root="data",
    batch_size=256,
    num_workers=8,
)
print(SingleEpochEval(model, train_loader, criterion, "cuda", 50))

# for name, module in model.named_modules():
#     print(name, module)



In [12]:
# %%
for name, module in model.named_modules():
    if (
        len(name) > 2
        and name[-7:] == "through"
        and hasattr(module, "activation_post_process")
    ):
        # if name == "act_obs":
        print(f"{name}", end=" ")

        _hist = list()
        print(module.activation_post_process.histogram.shape)
        tmp = module.activation_post_process.histogram.to("cpu").numpy()
        # for x in module.activation_post_process.get_tensor_value():
        #     _hist.append(torch.tensor(x).to("cpu"))

        #     print(torch.tensor(x).to("cpu").numpy().shape, end=" ")

        # _hist = np.array(_hist).flatten()

        # cnt = 1
        # for i in _hist.shape:
        #     cnt *= i

        # tmp = cnt * 4 / 1024 / 1024, "MB"
        # print(tmp)
        print(module.activation_post_process.min_val)
        print(module.activation_post_process.max_val)
        print("")
        # plt.hist(tmp)
        # plt.title(name)
        # # plt.yscale("log")
        # plt.xlabel("Activation value")
        # plt.ylabel("Frequency")
        # plt.show()

layer1.0.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(39.0166, device='cuda:0')

layer1.1.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(37.5907, device='cuda:0')

layer1.2.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(137.5625, device='cuda:0')

layer2.0.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(97.2317, device='cuda:0')

layer2.1.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(189.9802, device='cuda:0')

layer2.2.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(190.2122, device='cuda:0')

layer2.3.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(167.3941, device='cuda:0')

layer3.0.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(103.5171, device='cuda:0')

layer3.1.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(70.9293, device='cuda:0')

layer3.2.through torch.Size([2048])
tensor(0., device='cuda:0')
tensor(57.2622, device='cuda:0')

layer3.3.throug